<a href="https://colab.research.google.com/github/shruti-sriniv/Prompt-Chaining/blob/main/prompt_chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [1]:
!pip -q install --upgrade openai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 15.0 MB/s eta 0:00:00


In [10]:
from google.colab import userdata

In [15]:
import os
from typing import Optional, Dict

GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

print("Gemini key set? ", bool(GEMINI_API_KEY))

Gemini key set?  True


Creating a function to call the Model and provide a response

In [16]:
from dataclasses import dataclass

@dataclass
class ModelProvider:
  name: str
  model: str

# Choosing a default model
DEFAULT = ModelProvider("gemini", "gemini-1.5-flash")

# Defining a function to call the LLM
def call_model(prompt: str, provider : Optional[ModelProvider] = None, extra: Optional[Dict]=None) -> str:
  provider = provider or DEFAULT
  extra = extra or {}

  if provider.name.lower() == "gemini":
    if not GEMINI_API_KEY:
      raise RuntimeError("Install Gemini client: 'pip install google-generativeai'") from e

      genai.configure(GEMINI_API_KEY)
      model = genai.GenerativeModel(provider.model)
      response = model.generate_content(prompt)
      return response.text
  else:
    raise ValueError(f"Unknown provider: {provider.name}")


Providing Input